In [1]:
import json
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")
pd.set_option("display.max_rows", None, "display.max_columns", None)
from extraction import extract 


In [2]:
ext = extract.get_instance()

In [53]:
def extracter(text, ob_name):
    try:
        return ','.join(sorted(ext.extract_roles_from_label(text)[ob_name]))
    except:
        return ''

In [3]:
FILES = ['Admin', 'Kiran', 'Henrik']

In [4]:


df_all=[]

for file in FILES:
    #read
    f=open(f'../../src/datasets/datasets for tagging/dropbox_henrik/labeld/{file}.json', encoding="utf8")
    meta_data = json.load(f)
    f.close()
    
    #create and manipulate df
    df=pd.DataFrame(meta_data)
    df=df[['data','label','process model']]
    df['verbs'] = df.data.apply(lambda x: extracter(x,'action:name'))
    df['object'] = df.data.apply(lambda x: extracter(x,'object:name'))
    #df['adpositions'] = df.data.apply(lambda x: get_adpositions(x))
    df['label'] = df.label.apply(lambda x: str(sorted(x)))
    df['user'] = file
    
    print(f'----------------------- {file} ----')
    print(df['label'].value_counts())
    
    #concat
    result = df.to_json(orient="records")
    parsed = json.loads(result)
    df_all = df_all+parsed
    

df_all=pd.DataFrame(df_all)

----------------------- Admin ----
['Physical activity', 'User activity']                          225
['Automated activity', 'Physical activity', 'User activity']     66
['Physical activity']                                            42
['User activity']                                                18
['Automated activity', 'User activity']                          10
['Other']                                                         5
Name: label, dtype: int64
----------------------- Kiran ----
['Physical activity', 'User activity']                          275
['Physical activity']                                            45
['User activity']                                                28
['Automated activity', 'Physical activity', 'User activity']     10
['Other']                                                         5
['Other', 'Physical activity', 'User activity']                   1
['Automated activity', 'User activity']                           1
Name: label, dtype: 

In [83]:
df_inconsistency = pd.pivot_table(df_all,values='label',index=['data', 'process model', 'verbs', 'object'],aggfunc=lambda x:list(set(x.values))).reset_index()
df_inconsistency['number of missmatches']=df_inconsistency['label'].apply(lambda x:len(x))
df_inconsistency=df_inconsistency[df_inconsistency['number of missmatches']>1]
df_inconsistency.reset_index(drop=True,inplace=True)
df_inconsistency.head()

,data,process model,verbs,object,label,number of missmatches
0,Advertise post,Model8-1.svg,advertise,post,"[['Physical activity'], ['Physical activity', ...",2
1,Alert supervisor,Model9-1.svg,alert,,"[['Physical activity', 'User activity'], ['Aut...",2
2,Approve,Model5-1.svg,approve,,"[['Physical activity', 'User activity'], ['Aut...",2
3,Assign order to waiter,hotelservice.svg,assign,order,"[['Physical activity', 'User activity'], ['Aut...",2
4,Attach new SCT document,Model3-1.svg,attach,new sct document,"[['Physical activity'], ['Physical activity', ...",2


In [84]:
df_object_rel=pd.DataFrame({'object':df['object'].value_counts(normalize=True).index,'object: relative frequencies (total)':df['object'].value_counts(normalize=True).values})
df_object_abs=pd.DataFrame({'object':df['object'].value_counts(normalize=True).index,'object: absolut (total)':df['object'].value_counts().values})
df_object_abs=df_object_abs.merge(df_object_rel,on='object')
df_object_abs.head()

,object,object: absolut (total),object: relative frequencies (total)
0,,34,0.092896
1,order,11,0.030055
2,bill,6,0.016393
3,dismissal,5,0.013661
4,job description,4,0.010929


In [85]:
df_verbs_rel=pd.DataFrame({'verbs':df['verbs'].value_counts(normalize=True).index,'verb: relative frequencies (total)':df['verbs'].value_counts(normalize=True).values})
df_verbs_abs=pd.DataFrame({'verbs':df['verbs'].value_counts(normalize=True).index,'verb: absolut (total)':df['verbs'].value_counts().values})
df_verbs_abs=df_verbs_abs.merge(df_verbs_rel,on='verbs')
df_verbs_abs.head()

,verbs,verb: absolut (total),verb: relative frequencies (total)
0,inform,23,0.062842
1,create,19,0.051913
2,check,19,0.051913
3,confirm,17,0.046448
4,reject,16,0.043716


In [86]:
df_inconsistency=df_inconsistency.merge(df_verbs_abs,on='verbs',how='left')
df_inconsistency=df_inconsistency.merge(df_object_abs,on='object',how='left')
df_inconsistency.head()

,data,process model,verbs,object,label,number of missmatches,verb: absolut (total),verb: relative frequencies (total),object: absolut (total),object: relative frequencies (total)
0,Advertise post,Model8-1.svg,advertise,post,"[['Physical activity'], ['Physical activity', ...",2,2,0.005464,3,0.008197
1,Alert supervisor,Model9-1.svg,alert,,"[['Physical activity', 'User activity'], ['Aut...",2,2,0.005464,34,0.092896
2,Approve,Model5-1.svg,approve,,"[['Physical activity', 'User activity'], ['Aut...",2,4,0.010929,34,0.092896
3,Assign order to waiter,hotelservice.svg,assign,order,"[['Physical activity', 'User activity'], ['Aut...",2,6,0.016393,11,0.030055
4,Attach new SCT document,Model3-1.svg,attach,new sct document,"[['Physical activity'], ['Physical activity', ...",2,1,0.002732,1,0.002732


In [89]:
#df_inconsistency.to_excel('../../src/datasets/df_inconsistency.xlsx',index=False)

In [91]:
df_object_abs.to_excel('../../src/datasets/df_objects.xlsx',index=False,sheet_name='objects')
df_verbs_abs.to_excel('../../src/datasets/df_verbs.xlsx',index=False,sheet_name='verbs')

In [95]:
df_basic=df_all.merge(df_verbs_abs,on='verbs',how='left')
df_basic=df_basic.merge(df_object_abs,on='object',how='left')
df_basic.head()

,data,label,process model,verbs,object,user,verb: absolut (total),verb: relative frequencies (total),object: absolut (total),object: relative frequencies (total)
0,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,retrieve,sct file,Admin,2,0.005464,2,0.005464
1,Distribute SCT warrant possession,"['Physical activity', 'User activity']",Model3-1.svg,distribute,sct warrant possession,Admin,2,0.005464,1,0.002732
2,Store SCT physical file,['Physical activity'],Model3-1.svg,store,sct physical file,Admin,3,0.008197,2,0.005464
3,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,retrieve,sct file,Admin,2,0.005464,2,0.005464
4,Attach new SCT document,"['Physical activity', 'User activity']",Model3-1.svg,attach,new sct document,Admin,1,0.002732,1,0.002732


In [96]:
#df_basic.to_excel('../../src/datasets/df_basic.xlsx',index=False)

###### obsolete #######
def get_verbs(text):
    doc = nlp(text)
    return [token.text for token in doc if token.pos_ == 'VERB']

def get_nouns(text):
    text=' '.join(text.split()[1:])
    doc = nlp(text)
    return ', '.join(sorted([token.text for token in doc if (token.pos_ == 'NOUN') ]))#or (token.pos_ == 'PRON') or (token.pos_ == 'PROPN')

def get_adpositions(text):
    text=' '.join(text.split()[1:])
    doc = nlp(text)
    return ', '.join(sorted([token.text for token in doc if token.pos_ == 'ADP']))

df_all=[]

for file in FILES:
    #read
    f=open(f'../../src/datasets/datasets for tagging/dropbox_henrik/labeld/{file}.json', encoding="utf8")
    meta_data = json.load(f)
    f.close()
    
    #create and manipulate df
    df=pd.DataFrame(meta_data)
    df=df[['data','label','process model']]
    df['verbs'] = df.data.apply(lambda x: x.split()[0])
    df['nouns'] = df.data.apply(lambda x: get_nouns(x))
    df['adpositions'] = df.data.apply(lambda x: get_adpositions(x))
    df['label'] = df.label.apply(lambda x: str(sorted(x)))
    df['user'] = file
    
    print(f'----------------------- {file} ----')
    print(df['label'].value_counts())
    
    #concat
    result = df.to_json(orient="records")
    parsed = json.loads(result)
    df_all = df_all+parsed
    

df_all=pd.DataFrame(df_all)

In [13]:
df_all.head()

,data,label,process model,verbs,nouns,user
0,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,[retrieve],[sct file],Admin
1,Distribute SCT warrant possession,"['Physical activity', 'User activity']",Model3-1.svg,[distribute],[sct warrant possession],Admin
2,Store SCT physical file,['Physical activity'],Model3-1.svg,[store],[sct physical file],Admin
3,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,[retrieve],[sct file],Admin
4,Attach new SCT document,"['Physical activity', 'User activity']",Model3-1.svg,[attach],[new sct document],Admin


In [14]:
df_all.verbs.value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[inform]              69
[create]              57
[check]               57
[confirm]             51
[reject]              48
[conduct]             33
[send]                33
[prepare]             33
[notify]              29
[review]              27
[determine]           24
[register]            21
[update]              18
                      18
[request]             18
[assign]              18
[perform]             15
[test]                15
[transmit]            15
[report]              12
[process]             12
[give]                12
[examine]             12
[receive]             12
[approve]             12
[check for]           12
[ask for]              9
[enter]                9
[repair]               9
[store]                9
[select]               6
[deregister]           6
[sort]                 6
[distribute]           6
[capture]              6
[apply]                6
[advertise]            6
[import]               6
[meet]                 6
[plan]                 6


In [50]:
df[df['verbs']=='check']

,data,label,process model,verbs,nouns,adpositions,user
66,check open leads,"['Physical activity', 'User activity']",Model7-1.svg,check,,,Henrik
189,check invoice items,"['Physical activity', 'User activity']",Model6-1.svg,check,"invoice, items",,Henrik
267,check and suggest updates,"['Physical activity', 'User activity']",Model9-3.svg,check,,,Henrik
269,check for conflicts,"['Physical activity', 'User activity']",Model9-3.svg,check,conflicts,for,Henrik
297,check CRM systems,['User activity'],Model9-1.svg,check,systems,,Henrik
347,check part quantity,"['Physical activity', 'User activity']",bicyclemanufacturing.svg,check,"part, quantity",,Henrik


In [35]:
df_all.nouns.value_counts()

                                                                         135
order                                                                     30
file                                                                      27
assignment                                                                24
dismissal                                                                 18
bill                                                                      18
design                                                                    15
description, job                                                          12
mail                                                                      12
equipment                                                                 12
change                                                                    12
registration                                                              12
failure, report                                                            9

In [49]:
df[df['nouns']=='file']

,data,label,process model,verbs,nouns,adpositions,user
0,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,Retrieve,file,,Henrik
2,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Henrik
3,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,Retrieve,file,,Henrik
5,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Henrik
94,Print physical file,"['Automated activity', 'Physical activity', 'U...",Model3-5.svg,Print,file,,Henrik
115,Compose file,"['Physical activity', 'User activity']",claimscreation.svg,Compose,file,,Henrik
211,Create patient file,"['Physical activity', 'User activity']",Model4-1.svg,Create,file,,Henrik
215,Ask for medical file,"['Physical activity', 'User activity']",Model4-1.svg,Ask,file,for,Henrik
216,Update patient file,"['Physical activity', 'User activity']",Model4-1.svg,Update,file,,Henrik


In [36]:
df_all.head(15)

,data,label,process model,verbs,nouns,adpositions,user
0,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,Retrieve,file,,Admin
1,Distribute SCT warrant possession,"['Physical activity', 'User activity']",Model3-1.svg,Distribute,"possession, warrant",,Admin
2,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Admin
3,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,Retrieve,file,,Admin
4,Attach new SCT document,"['Physical activity', 'User activity']",Model3-1.svg,Attach,document,,Admin
5,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Admin
6,Bring in defective computer,['Physical activity'],Model1-2.svg,Bring,computer,in,Admin
7,Prepare repair cost calculation,"['Physical activity', 'User activity']",Model1-2.svg,Prepare,"calculation, cost, repair",,Admin
8,Receive repair cost calculation,"['Physical activity', 'User activity']",Model1-2.svg,Receive,"calculation, cost, repair",,Admin
9,Take computer home,['Physical activity'],Model1-2.svg,Take,"computer, home",,Admin


In [37]:
df_all[(df_all['label']=="['Physical activity']") & (df_all['user']=="Henrik")].nouns.value_counts()

                              4
mail                          3
file                          2
files                         2
equipment                     2
repair                        2
inspection                    1
bicycle                       1
count, installation, meter    1
vacancy                       1
quality                       1
order                         1
copy, invoice                 1
copy, docket, invoice         1
estimate, settlement          1
exhibition                    1
computer                      1
products                      1
room, service                 1
room                          1
beverages                     1
cellar, wine                  1
drinks                        1
cart                          1
meal                          1
computer, home                1
bicycle, customer             1
Name: nouns, dtype: int64

In [38]:
df_all[(df_all['label']=="['Physical activity', 'User activity']") & (df_all['user']=="Henrik")].nouns.value_counts()

                                                                40
order                                                            9
assignment                                                       8
file                                                             6
dismissal                                                        6
bill                                                             6
design                                                           5
description, job                                                 4
registration                                                     4
change                                                           3
part                                                             3
failure, report                                                  3
failure                                                          3
application                                                      3
check                                                         

In [39]:
df_all[(df_all['label']=="['Physical activity', 'User activity']") & (df_all['user']=="Henrik")].head()

,data,label,process model,verbs,nouns,adpositions,user
731,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,Retrieve,file,,Henrik
732,Distribute SCT warrant possession,"['Physical activity', 'User activity']",Model3-1.svg,Distribute,"possession, warrant",,Henrik
734,Retrieve SCT file,"['Physical activity', 'User activity']",Model3-1.svg,Retrieve,file,,Henrik
735,Attach new SCT document,"['Physical activity', 'User activity']",Model3-1.svg,Attach,document,,Henrik
738,Prepare repair cost calculation,"['Physical activity', 'User activity']",Model1-2.svg,Prepare,"calculation, cost, repair",,Henrik


In [40]:
df_all[(df_all['label']=="['Physical activity']") & (df_all['user']=="Kiran")]

,data,label,process model,verbs,nouns,adpositions,user
368,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Kiran
370,Attach new SCT document,['Physical activity'],Model3-1.svg,Attach,document,,Kiran
371,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Kiran
372,Bring in defective computer,['Physical activity'],Model1-2.svg,Bring,computer,in,Kiran
375,Take computer home,['Physical activity'],Model1-2.svg,Take,"computer, home",,Kiran
377,Check and repair hardware,['Physical activity'],Model1-2.svg,Check,hardware,,Kiran
378,Check and configure software,['Physical activity'],Model1-2.svg,Check,"configure, software",,Kiran
379,Test system functionality,['Physical activity'],Model1-2.svg,Test,"functionality, system",,Kiran
382,Track physical files,['Physical activity'],Model3-2.svg,Track,files,,Kiran
388,Prepare meal,['Physical activity'],hotelservice.svg,Prepare,meal,,Kiran


In [41]:
df_all[(df_all['label']=="['Physical activity']") & (df_all['user']=="Henrik")]

,data,label,process model,verbs,nouns,adpositions,user
733,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Henrik
736,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Henrik
737,Bring in defective computer,['Physical activity'],Model1-2.svg,Bring,computer,in,Henrik
740,Take computer home,['Physical activity'],Model1-2.svg,Take,"computer, home",,Henrik
747,Track physical files,['Physical activity'],Model3-2.svg,Track,files,,Henrik
748,Hand files to associate,['Physical activity'],Model3-2.svg,Hand,files,,Henrik
753,Prepare meal,['Physical activity'],hotelservice.svg,Prepare,meal,,Henrik
755,Ready cart,['Physical activity'],hotelservice.svg,Ready,cart,,Henrik
756,Prepare non-alcoholic drinks,['Physical activity'],hotelservice.svg,Prepare,drinks,,Henrik
758,Fetch wine from cellar,['Physical activity'],hotelservice.svg,Fetch,"cellar, wine",from,Henrik


In [42]:
df_all[(df_all['label']=="['Physical activity']") & (df_all['user']=="Kiran")]

,data,label,process model,verbs,nouns,adpositions,user
368,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Kiran
370,Attach new SCT document,['Physical activity'],Model3-1.svg,Attach,document,,Kiran
371,Store SCT physical file,['Physical activity'],Model3-1.svg,Store,file,,Kiran
372,Bring in defective computer,['Physical activity'],Model1-2.svg,Bring,computer,in,Kiran
375,Take computer home,['Physical activity'],Model1-2.svg,Take,"computer, home",,Kiran
377,Check and repair hardware,['Physical activity'],Model1-2.svg,Check,hardware,,Kiran
378,Check and configure software,['Physical activity'],Model1-2.svg,Check,"configure, software",,Kiran
379,Test system functionality,['Physical activity'],Model1-2.svg,Test,"functionality, system",,Kiran
382,Track physical files,['Physical activity'],Model3-2.svg,Track,files,,Kiran
388,Prepare meal,['Physical activity'],hotelservice.svg,Prepare,meal,,Kiran


In [43]:
df.verbs.value_counts()

Inform         22
Check          20
Confirm        17
Create         16
Reject         15
Send           13
Notify         10
Prepare        10
Review          9
Determine       8
Request         6
Conduct         6
Assign          6
check           6
Perform         5
Update          5
Register        5
Test            5
conduct         5
Transmit        5
Approve         4
Store           4
Report          4
Examine         4
Ask             3
Give            3
create          3
Capture         2
Deregister      2
Process         2
Enter           2
Sort            2
Retrieve        2
Import          2
Distribute      2
Plan            2
Meet            2
register        2
repair          2
Advertise       2
Apply           2
Alert           2
Acquire         2
Change          2
Remind          2
receive         2
Undertake       2
process         2
Verify          2
Receive         2
Invoke          2
Return          2
Track           2
Take            2
Hand            2
select    

# FINAL

In [39]:
df_resolution = pd.read_excel('../../src/datasets/df_inconsistency_Henrik.xlsx')[['data','process model','resolution']]
df_resolution=df_resolution.rename(columns={'resolution':'label'})
df_resolution.head(2)

,data,process model,label
0,Advertise post,Model8-1.svg,"['Physical activity', 'User activity']"
1,Alert supervisor,Model9-1.svg,"['Automated activity', 'Physical activity', 'U..."


In [40]:
df_resolution.tail(2)

,data,process model,label
136,reserve part,bicyclemanufacturing.svg,"['Automated activity', 'Physical activity', 'U..."
137,update group calendars,Model9-3.svg,"['Physical activity', 'User activity']"


In [36]:
df_all = pd.read_excel('../../src/datasets/df_basic.xlsx')[['data','process model','label']]
df_all=pd.pivot_table(df_all,values='label',index=['data', 'process model'],aggfunc=lambda x:list(set(x.values))).reset_index()
df_all['number of missmatches']=df_all['label'].apply(lambda x:len(x))
df_all=df_all[df_all['number of missmatches']==1]

df_all.reset_index(drop=True,inplace=True)
df_all['label']=df_all['label'].apply(lambda x:x[0])
df_all=df_all.drop('number of missmatches',axis=1)
df_all.tail()

,data,process model,label
211,offer to public,Model1-4.svg,"['Physical activity', 'User activity']"
212,register,repairexample.svg,"['Physical activity', 'User activity']"
213,select top five leads,Model7-1.svg,"['Physical activity', 'User activity']"
214,select unchecked part,bicyclemanufacturing.svg,"['Physical activity', 'User activity']"
215,ship bicycle to customer,bicyclemanufacturing.svg,['Physical activity']


In [54]:
df_final = pd.concat([df_resolution, df_all],ignore_index=True)
df_final['data']=df_final['data'].apply(lambda x: x.lower())
df_final = df_final.drop('process model',axis=1).drop_duplicates(ignore_index=True).sort_values('data',ignore_index=True)
#df_final['verbs'] = df_final.data.apply(lambda x: extracter(x,'action:name'))
#df_final['object'] = df_final.data.apply(lambda x: extracter(x,'object:name'))
df_final['object:name'] = df_final.data.apply(lambda x: extracter(x,'object:name'))
df_final['object:instance'] = df_final.data.apply(lambda x: extracter(x,'object:instance'))
df_final['object:status'] = df_final.data.apply(lambda x: extracter(x,'object:status'))
df_final['action:name'] = df_final.data.apply(lambda x: extracter(x,'action:name'))
df_final['action:status'] = df_final.data.apply(lambda x: extracter(x,'action:status'))
df_final['org:actor:name'] = df_final.data.apply(lambda x: extracter(x,'org:actor:name'))
df_final['org:actor:instance'] = df_final.data.apply(lambda x: extracter(x,'org:actor:instance'))
df_final['org:passive:name'] = df_final.data.apply(lambda x: extracter(x,'org:passive:name'))
df_final['org:passive:instance'] = df_final.data.apply(lambda x: extracter(x,'org:passive:instance'))
df_final

,data,label,object:name,object:instance,object:status,action:name,action:status,org:actor:name,org:actor:instance,org:passive:name,org:passive:instance
0,accept order,"['Automated activity', 'Physical activity', 'U...",order,,,accept,,,,,
1,acquire equipment,"['Physical activity', 'User activity']",equipment,,,acquire,,,,,
2,advertise post,"['Physical activity', 'User activity']",post,,,advertise,,,,,
3,alert supervisor,"['Automated activity', 'Physical activity', 'U...",,,,alert,,,,supervisor,
4,analyze defect,"['Physical activity', 'User activity']",defect,,,analyze,,,,,
5,analyze problem causes in detail,"['Physical activity', 'User activity']",problem causes detail,,,analyze,,,,,
6,answer notice,"['Physical activity', 'User activity']",notice,,,answer,,,,,
7,apply,"['Physical activity', 'User activity']",,,,apply,,,,,
8,approve,"['Automated activity', 'Physical activity', 'U...",,,,approve,,,,,
9,approve job description,"['Physical activity', 'User activity']",job description,,,approve,,,,,


In [55]:
df_final.to_csv('../../src/datasets/01_basic_dataset_for_classification.csv')